In [1]:
import requests

In [2]:
url = "https://www.worldometers.info/coronavirus"

In [3]:
req = requests.get(url)

In [4]:
req

<Response [200]>

In [5]:
from lxml.html import fromstring

In [6]:
root = fromstring(req.text)

In [7]:
table = root.xpath(".//table[@id='main_table_countries_today'][1]")[0]

In [8]:
table

<Element table at 0x7efbe832d830>

In [9]:
thead = table.xpath(".//thead/tr/th")

In [10]:
def is_visible(attr):
    if 'style' in attr.attrib:
        style = attr.get('style')
        if style.__contains__("display:none") or style.__contains__("display: none"):
            return False
    return True

In [11]:
columns = list()

In [12]:
for th in thead:
    if is_visible(th) is True:
        columns.append(th.text_content().replace('\n', '').replace(u'\xa0', u'').strip())

In [13]:
columns

['#',
 'Country,Other',
 'TotalCases',
 'NewCases',
 'TotalDeaths',
 'NewDeaths',
 'TotalRecovered',
 'NewRecovered',
 'ActiveCases',
 'Serious,Critical',
 'TotCases/1M pop',
 'Deaths/1M pop',
 'TotalTests',
 'Tests/1M pop',
 'Population',
 '1 Caseevery X ppl',
 '1 Deathevery X ppl',
 '1 Testevery X ppl']

In [14]:
tbody = table.xpath(".//tbody/tr")

In [15]:
tags = list()
values = list()

In [16]:
for tr in tbody:
    if is_visible(tr) is True:
        tag = tr.xpath(".//td/a")
        if len(tag) > 0:
            tags.append(tag[0].get('href').strip().replace('country/', '').replace('/', ''))

        values.append([value.text_content().strip() for value in tr.xpath(".//td") if is_visible(value) is True])

In [17]:
import pandas as pd

In [18]:
data = pd.DataFrame(values, columns=columns)

In [19]:
data.head()

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",TotCases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,Population,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl
0,,World,"10,169,777","+94,662","502,809","+2,185","5,508,727","+54,179","4,158,241","57,949","1,305",64.5,,,,,,
1,1,USA,"2,617,847","+21,310","128,243",+91,"1,082,212",+775,"1,407,392","15,832","7,909",387,"32,143,196","97,113","330,986,397",126,"2,581",10
2,2,Brazil,"1,319,274","+3,333","57,149",+46,"715,905",,"546,220","8,318","6,207",269,"2,925,935","13,766","212,545,812",161,"3,719",73
3,3,Russia,"634,437","+6,791","9,073",+104,"399,087","+5,735","226,277","2,300","4,347",62,"19,044,954","130,504","145,934,108",230,"16,084",8
4,4,India,"548,857","+19,280","16,485",+382,"321,766","+11,620","210,606","8,944",398,12,"8,227,802","5,963","1,379,863,384","2,514","83,704",168


In [20]:
data.to_csv('../data/overview.csv')